In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipynb.fs.defs.tdc_data_analysis import line_hist

**TODO**: Update wherever possible from pandas frames to numpy

In [3]:
def tdc_stripper(read_filename, write_filename,total_lines = -1):
    line_counter = 0
    with open(read_filename,'r') as rfile:
        with open(write_filename,'w') as wfile:
            for line in rfile:
                line_counter+=1
                if line_counter == total_lines:
                    break
                if line[:3] == 'TDC':
                    wfile.write(line)
                

In [13]:
tdc_stripper('data_2021_06_10/converted_xe_2tdc_45deg_11.txt','45deg_11_tdc.txt')

In [15]:
_,tmp,tmp2,_= preproces_tdcs('xe_vert_4.txt')

tpx_df nulls
 tdc     0
time    0
dtype: int64


In [16]:
print(len(tmp),len(tmp2))

319169 294045


In [7]:
def preproces_tdcs(filename=None,debug=True,tdc_df=None):
    ''' 
    Args:
        filename : string
            name of the file
        debug : bool
              if True returns all dataframes
        tdc_df : pd.DataFrame Object
            instead of reading from file one can give the df directly
    Returns:
        time_df : pd.DataFrame Object
            
    It takes the data file for TDC's ONLY, which is assumed to be in the form
    TDCn: t0.
    We first seperate TDC's apart and then merge them, which creates nulls.
    The nulls for TDC2 are back filled i.e. if a cell N is null then it's filled 
    with the last cell <N which isn't null. TDC1 nulls aren't filled but rather dropped
    because we are really interested in TDC2. Finally we calculate the time difference
    between TD1 and TDC2. Finally we gate the difference by 5*10^4 ns = 50 us.
    '''
    if filename is not None:
        tpx_df =pd.read_csv(filename,
                    sep=':',
                    error_bad_lines=False,
                    warn_bad_lines=False,
                    header=None)
    elif tdc_df is not None:
        tpx_df = tdc_df
    else:
        raise(Exception('Either a filename or a tdc data frame needs to be given!'))
    tpx_df.columns = ['tdc','time']

    print('tpx_df nulls\n',tpx_df.isna().sum())
    tdc_one = tpx_df[tpx_df.tdc == 'TDC1']
    tdc_two = tpx_df[tpx_df.tdc == 'TDC2']
    time_df = tdc_one[['time']].join(tdc_two.time,how='outer',lsuffix='1',rsuffix='2')
    time_df['time2'] = time_df['time2'].fillna(method='bfill')
    time_df['time1'] = time_df['time1'].fillna(method='bfill')

    #time_df.dropna(inplace=True)
    time_df = time_df.astype(np.float64)
    time_df['delta_t'] = (time_df.time2-time_df.time1)*1e9
    time_df.reset_index(inplace=True)
    time_df.drop('index',axis=1,inplace=True)
    #time_df = time_df[np.abs(time_df.delta_t)< 5e3]
    if debug:
        return tpx_df, tdc_one, tdc_two, time_df
    else:
        print(time_df.head(10))
        return time_df

In [14]:
_,tdc_one, tdc_two,_= preproces_tdcs('45deg_11_tdc.txt',debug=True)
tdc_one = np.array(tdc_one['time'])
tdc_two = np.array(tdc_two['time'])


NTOTBATCHES = 10_000    # Max number of batches to process -1=inf
NBATCH = 2_000    # Number of batches to collect before writing it to the file
BEGINLINE= 0   # starting line, used mainly for debugging
line_counter = 0
NLINES = -1  # Max number of lines to be read -1=inf
batch = 0

tdc_updated = False

t1_eps = 9e-4 #tdc1 tolerance [s]
t2_eps = 3e-6 #tdc2 tolerance [s]

x_threshold = 15  # max batch size in x direction for rejecting many electrons
y_threshold = 15 # max batch size in y direction for rejecting many electrons
xmax = 0
xmin = 1000 # detector has only 256x256 pixels
ymax = 0
ymin = 1000 # detector has only 256x256 pixels

batch_lines = ''
df_str = ''

many_electron_reject = 0
tdc1_reject = 0
tdc2_reject= 0

# We gate tdc2-tdc1 differnce (which corresponds to cable length)
# and reject if this difference is too much or too little
# physical_max/min_t's were determined by an preliminary analysis of the data
# and they are given in ns.

unphysical_reject = 0

physical_max_t = 4795   # max allowed tdc2-tdc1 difference [ns]
physical_min_t = 4785   # min allowed tdc2-tdc1 difference [ns]

toa_overflow_counter = 0
tdc1_overflow_counter = 0
tdc2_overflow_counter = 0

toa_overflow_threshold = (25*2**30/1e9)   # when pixel's toa overflows 
tdc_overflow_threshold = (25*2**32/1e9)  # when tdcs overflow

tdc1_overflow = False
tdc2_overflow = False
toa_overflow = False

overflow_threshold = 10     # sets after which we start checking overflow
                            # i.e. difference bw subsequent values need to
                            # be at least this threshold

toa = 0



tdc1_iter = tdc_one.flat   # flat produces an iterator from np.array
tdc1 = next(tdc1_iter)
tdc1_next = next(tdc1_iter)
tdc1_batch = 0 # chosen tdc1 for the batch


tdc2_iter = tdc_two.flat   # flat produces an iterator from np.array
tdc2 = next(tdc2_iter)
tdc2_next = next(tdc2_iter)
tdc2_batch = 0 # chosen tdc2 for the batch


header = 'batch,x,y,delta_t\n'



with open('preprocessed110.csv','w') as wfile:
    wfile.write('')

import re
re_search = re.compile('\w+:')
with open('data_2021_06_10/converted_xe_2tdc_45deg_11.txt','r') as rfile:
    with open('preprocessed110.csv','a') as afile:
        afile.write(header)
        
        def overflow_corr(num,num_counter,num_threshold): #corrects for overflow
            return num + num_counter*num_threshold
        
        for line in rfile:
            line_counter+=1
            if line_counter < BEGINLINE:
    #            toa_overflow_counter =3 #for 12_000_000
                continue
                
            if line_counter %1_000_000 == 0:
                print(f'{line_counter:_} lines read')

            
            if batch % NBATCH == 0 and batch>0 and df_str != '':
                df_str='toa,tot,x,y,tdc1,tdc2,batch\n'+df_str
                to_write = final_processing(df_str)
                afile.write(to_write)
                to_write = ''
                #break
                df_str = ''
                print(f'\nBatch numer {batch:_} is written')
                print(f'{line_counter:_} lines read')
                print('-----')


                
            if batch == NTOTBATCHES:
                print('NBATCHES reached')
                break
            if line_counter == NLINES:
                print('NLINES reached')
                break

                    
############################################ TOA #######################################
                
                
            str_list = line.split('TOA:')
            try:
                toa_prev = toa
                toa = np.float64(str_list[1].split(',')[0])
                toa = overflow_corr(toa,toa_overflow_counter,toa_overflow_threshold)
                    
                if toa_prev > toa + overflow_threshold:
                    
                    toa_overflow_counter += 1
                    #print('toa_prev %s toa %s toa_prev-toa %s' % (toa_prev,toa,toa_prev-toa))
                    toa += toa_overflow_threshold
                    print('### TOA Overflow NO %s' %toa_overflow_counter )
                    print('toa_prev %s toa %s tdc1 %s tdc2 %s'  % (toa_prev,toa,tdc1,tdc2))
                    

                
            except IndexError: # This only happens for tdc lines
                continue

                
############################################ TDC2  #######################################

                
            # tdc2 iterator is only updated when the difference of toa with the current tdc2
            # is greater than the difference of toa with the next tdc2
            while abs(toa-tdc2) >= abs(toa-tdc2_next):
                tdc2 = tdc2_next
                tdc2_next = next(tdc2_iter)

                
                # Handle overflow
                tdc2_next = overflow_corr(tdc2_next,tdc2_overflow_counter,tdc_overflow_threshold)
                
                if tdc2 > tdc2_next+overflow_threshold:
                   # print('tdc2 %s tdc2_next %s tdc1 %s toa %s' % (tdc2,tdc2_next ,tdc1,toa))

                    tdc2_overflow_counter += 1
                    tdc2_next += tdc_overflow_threshold
                    print('### TDC2 Overflow NO %s' %tdc2_overflow_counter )
                    print('tdc2 %s tdc2_next %s tdc1 %s toa %s' % (tdc2,tdc2_next ,tdc1,toa))


                    

                    
                tdc_updated = True
    
    
############################################ MANY ELECTRON  #######################################

            # We want to gate >1 electron events. This is achieved by putting a max-min threshold
            # on x and y. 
        
            # before we deal with many electrons we want to reject unphysical events. these are
            # due to e.g. dark counts and faults of the detector
            
            if tdc_updated:
                # *1e9 is to convert from s to ns
                if ((tdc2_batch-tdc1_batch)*1e9 < physical_max_t and \
                    (tdc2_batch-tdc1_batch)*1e9 > physical_min_t) or True:
                    if abs(xmax-xmin) < x_threshold and abs(ymax-ymin) < y_threshold:
                        df_str = df_str + batch_lines
                        batch += 1
                    else:
                        if np.random.rand()<1/1250:   # arbitrary number that determines when to plot pics
                            many_e = pd.read_csv(StringIO('toa,tot,x,y,tdc1,tdc2,batch\n'+batch_lines))
                            try:
                                many_e.plot(x='x',y='y',linestyle='None',marker='o')
                                plt.show()
                            except:
                                None
                        many_electron_reject +=1
                        if many_electron_reject % 5_000==0:
                            print(f'{many_electron_reject:_} physical batches were \
                                  rejected due to many electrons')
                else:
                    unphysical_reject += 1
                    if unphysical_reject % 5_000 == 0:
                        print(f'{unphysical_reject:_} batches were rejected due time gating')
                        

                    
                tdc_updated = False
                batch_lines =''
                xmax = 0
                xmin = 1000
                ymax = 0
                ymin = 1000
                
############################################ TDC1  #######################################


            # when the difference bw tdc1 and toa exceeds the threshold we need to update tdc1
            while toa-tdc1>=t1_eps:
                tdc1 = tdc1_next
                tdc1_next = next(tdc1_iter)
                
                # Handle overflow
                tdc1_next = overflow_corr(tdc1_next,tdc1_overflow_counter,tdc_overflow_threshold)
                
                if tdc1 > tdc1_next + overflow_threshold:
                    tdc1_overflow_counter += 1
                   # print('tdc1 %s tdc1_next %s tdc2 %s toa %s' % (tdc1,tdc1_next ,tdc2,toa))

                    tdc1_next += tdc_overflow_threshold
                    print('### TDC1 Overflow NO %s' %tdc1_overflow_counter )
                    print('tdc1 %s tdc1_next %s tdc2 %s toa %s' % (tdc1,tdc1_next ,tdc2,toa))


                
                
            if toa-tdc1 >=0 and toa-tdc1<t1_eps:  
                # We assume that tdc1 signal must be the first signal hence >0
                # if the difference between time of arrival (toa) and tdc1 signal
                # is below threshold and toa-tdc2 is below threshold we write into the file
                if abs(toa-tdc2) < t2_eps:
                    line = ''.join(re_search.split(line))[:-1] + (', %s , %s , %s\n' % (tdc1,tdc2,batch))
                    tdc1_batch = tdc1
                    tdc2_batch = tdc2
                    batch_lines = batch_lines + line
                    x,y = line.split(',')[2:4]
                    x,y = int(x),int(y)
                    xmax = max(x,xmax)
                    xmin = min(x,xmin)
                    ymax = max(y,ymax)
                    ymin = min(y,ymin)
                else:
                    tdc2_reject +=1
                    if tdc2_reject %100_000 == 0:
                        print(f'{tdc2_reject:_} lines were rejected due to tdc2 mismatch')
            else:
                tdc1_reject += 1
                if tdc1_reject %100_000 == 0:
                        print(f'{tdc1_reject:_} lines were rejected due to tdc1 mismatch')
                        
print('\n #########______######### \n')





tpx_df nulls
 tdc     0
time    0
dtype: int64

Batch numer 2_000 is written
110_936 lines read
-----

Batch numer 4_000 is written
220_998 lines read
-----
### TOA Overflow NO 1
toa_prev 26.8413062765625 toa 26.866684656249998 tdc1 26.841301921875 tdc2 26.8328472122375

Batch numer 6_000 is written
326_743 lines read
-----

Batch numer 8_000 is written
435_508 lines read
-----
### TOA Overflow NO 2
toa_prev 53.686897059375 toa 53.692044809375 tdc1 53.6868927304687 tdc2 53.6868974645813

Batch numer 10_000 is written
542_114 lines read
-----
NBATCHES reached

 #########______######### 



In [1]:
from preprocessing import *

In [5]:
preprocess(
    tdc_file='45deg_11_tdc.txt',
    data_file='data_2021_06_10/converted_xe_2tdc_45deg_11.txt')

tpx_df nulls
 tdc     0
time    0
dtype: int64
NLINES reached

 #########______######### 



In [44]:
abs(0.004851078125-0.0048514109375)

3.3281250000002927e-07

In [45]:
abs(0.004851078125- 0.0057913515625)

0.0009402734374999999

In [27]:
all_counters = {'tdc1_reject':tdc1_reject,'tdc2_reject':tdc2_reject,
               'many_electron_reject':many_electron_reject,
               'tdc1_overflow_counter':tdc1_overflow_counter,
               'tdc2_overflow_counter':tdc2_overflow_counter,
               'unphysical_reject':unphysical_reject,
               'batch':batch}
for name,val in all_counters.items():
    print(f'{name}: {val:_} \n')

tdc1_reject: 423 

tdc2_reject: 3_200 

many_electron_reject: 4_293 

tdc1_overflow_counter: 0 

tdc2_overflow_counter: 0 

unphysical_reject: 0 

batch: 10_000 



In [36]:
for name,val in all_counters.items():
    print(f'{name}: {val:_} \n')

tdc1_reject: 68_201 

tdc2_reject: 173_222 

many_electron_reject: 117 

tdc1_overflow_counter: 0 

tdc2_overflow_counter: 0 

unphysical_reject: 21_396 



In [95]:
tmp =  pd.read_csv('new_background11.csv')
tmp.delta_t = tmp.delta_t*1e9

In [96]:
tmp.describe()

,batch,x,y,delta_t
count,110000.000000,110000.000000,110000.000000,110000.000000
mean,54999.500000,95.933292,100.301015,4790.052696
std,31754.409143,32.648802,28.162456,0.980316
min,0.000000,43.074919,24.296113,4785.418696
25%,27499.750000,70.558930,80.337601,4789.324976
50%,54999.500000,90.675468,98.975940,4790.100093
75%,82499.250000,116.983751,119.817289,4790.624999
max,109999.000000,214.823393,195.395987,4794.793995


In [ ]:
tmp2 = pd.read_csv('preprocessed11.csv')
tmp2.info()

In [ ]:
tmp2=final_processing(tmp2,debug=True)

In [ ]:
x,y=line_hist(tmp.delta_t*1e9,bins=np.linspace(4785,4795,105),normalized=False,linestyle='None')

In [ ]:
(x*y).sum()/y.sum()

In [ ]:
x,y=line_hist(tmp.delta_t*1e9,bins=np.linspace(0,10000,5000),normalized=False,linestyle='None')
ycs = np.cumsum(y)
plt.plot(x,ycs/ycs[-1])

In [ ]:
(tmp['delta_t']*1e9<4795).sum()

In [ ]:
sns.displot(data=tmp[tmp['delta_t']*1e9<4795],x='x',y='y',bins=100)

In [ ]:
tmp.loc[26_715].toa1

In [ ]:
np.longdouble?

In [ ]:
print(final_processing(df_str))

In [ ]:
pd.read_csv('preprocessed110.csv')

In [ ]:
np.finfo('longdouble')

In [ ]:
np.finfo('float64')

In [3]:
tmp = 2
tmp2 = 3
tmp3  = [tmp,tmp2]
tmp2 = 4
tmp3

[2, 3]